## 1. Corner point detection

### 1.1 cv2.preCornerDetect()

In [1]:
import cv2
import numpy as np
#1
def findLocalMaxima(src):
    kernel= cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(11,11))
    dilate = cv2.dilate(src,kernel)# local max if kernel=None, 3x3 
    localMax = (src == dilate)
    
    erode = cv2.erode(src,kernel) #  local min if kernel=None, 3x3 
    localMax2 = src > erode      
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0, 1]] = points[:,[1, 0]] # switch x, y 
    return points

#2
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.preCornerDetect(gray, ksize=3)
ret, res2 = cv2.threshold(np.abs(res), 0.1, 0, cv2.THRESH_TOZERO)
corners = findLocalMaxima(res2)
print('corners.shape=', corners.shape)

#3
dst = src.copy()  
for x, y in corners:    
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)
    
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()

corners.shape= (8, 2)


### 1.2 cv2.cornerEigenValsAndVecs()

In [2]:
import cv2
import numpy as np
#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerEigenValsAndVecs(gray, blockSize=5, ksize=3)
print('res.shape=', res.shape)
eigen = cv2.split(res)

#2
T = 0.2
ret, edge = cv2.threshold(eigen[0], T, 255, cv2.THRESH_BINARY)
edge = edge.astype(np.uint8)

#3
corners = np.argwhere(eigen[1]>T)
corners[:,[0, 1]] = corners[:,[1, 0]] # switch x, y
print('len(corners) =', len(corners))

dst = src.copy()
for x, y in corners:  
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)
    
cv2.imshow('edge',  edge) 
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()

res.shape= (512, 512, 6)
len(corners) = 8


### 1.3 cv2.cornerMinEigenVal()

In [3]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
eigen = cv2.cornerMinEigenVal(gray, blockSize=5)
print('eigen.shape=', eigen.shape)

#2
T = 0.2
corners  = np.argwhere(eigen> T)
corners[:,[0, 1]] = corners[:,[1, 0]] # switch x, y
print('len(corners ) =', len(corners ))
dst = src.copy()
for x, y in corners :    
    cv2.circle(dst, (x, y), 3, (0,0,255), 2)
    
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()

eigen.shape= (512, 512)
len(corners ) = 8


### 1.4 cv2.cornerHarris(), cv2.cornerSubPix()

In [4]:
import cv2
import numpy as np

#1
def findLocalMaxima(src):
    kernel= cv2.getStructuringElement(shape=cv2.MORPH_RECT, ksize=(11,11))
    dilate = cv2.dilate(src,kernel)# local max if kernel=None, 3x3 
    localMax = (src == dilate)
    
    erode = cv2.erode(src,kernel) #  local min if kernel=None, 3x3 
    localMax2 = src > erode      
    localMax &= localMax2
    points = np.argwhere(localMax == True)
    points[:,[0, 1]] = points[:,[1, 0]] # switch x, y
    return points
#2
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.01)
ret, res = cv2.threshold(np.abs(res),0.02, 0, cv2.THRESH_TOZERO)
res8 = cv2.normalize(res, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imshow('res8',  res8)

corners = findLocalMaxima(res)
print('corners=', corners)

#3
#corners = np.float32(corners).copy()
corners = corners.astype(np.float32, order='C')
term_crit = (cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners,(5,5),(-1,-1), term_crit)
print('corners2=', corners2)

dst = src.copy()
for x, y in corners2:    
    cv2.circle(dst, (x, y), 3, (0,0,255), 2)
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()

corners= [[109 127]
 [264 127]
 [267 167]
 [386 170]
 [109 268]
 [167 271]
 [170 374]
 [386 374]]
corners2= [[107.55936  125.559456]
 [265.44077  125.559235]
 [265.55923  168.44174 ]
 [387.4408   168.55888 ]
 [107.55836  269.44095 ]
 [168.4415   269.5593  ]
 [168.5592   375.4408  ]
 [387.4408   375.4408  ]]


### 1.5 cv2.cornerHarris(), cv2.cornerSubPix()

In [5]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
res = cv2.cornerHarris(gray, blockSize=5, ksize=3, k=0.01)

#2
res = cv2.dilate(res, None) # 3x3 rect kernel
ret, res = cv2.threshold(res, 0.01*res.max(),255,cv2.THRESH_BINARY)
res8 = np.uint8(res)
cv2.imshow('res8',  res8)

#3
ret, labels, stats, centroids = cv2.connectedComponentsWithStats(res8)
print('centroids.shape=', centroids.shape)
print('centroids=',centroids)
centroids = np.float32(centroids)

#4
term_crit=(cv2.TERM_CRITERIA_MAX_ITER+cv2.TERM_CRITERIA_EPS,10, 0.001)
corners = cv2.cornerSubPix(gray, centroids, (5,5), (-1,-1), term_crit)
print('corners=',corners)

#5 
dst = src.copy()
for x, y in corners[1:]:    
    cv2.circle(dst, (x, y), 5, (0,0,255), 2)

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()

centroids.shape= (9, 2)
centroids= [[255.53481922 255.53107522]
 [108.         126.        ]
 [265.         126.        ]
 [266.         168.        ]
 [387.         169.        ]
 [108.         269.        ]
 [168.         270.        ]
 [169.         375.        ]
 [387.         375.        ]]
corners= [[255.53482  255.53108 ]
 [107.55842  125.55865 ]
 [265.44165  125.55835 ]
 [265.55832  168.44258 ]
 [387.44168  168.55795 ]
 [107.557495 269.44186 ]
 [168.4424   269.55838 ]
 [168.55832  375.44168 ]
 [387.44168  375.44168 ]]


### 1.6 cv2.goodFeaturesToTrack()

In [6]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/CornerTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

#K = 5 
K = 10
corners = cv2.goodFeaturesToTrack(gray, maxCorners=K,
              qualityLevel=0.05, minDistance=10)
print('corners.shape=',corners.shape)
print('corners=',corners)

#2
corners2 = cv2.goodFeaturesToTrack(gray, maxCorners=K,
               qualityLevel=0.05, minDistance=10,
               useHarrisDetector=True, k=0.04)
print('corners2.shape=',corners2.shape)
print('corners2=',corners2)

#3
dst = src.copy()
corners = corners.reshape(-1, 2)
for x, y in corners:    
    cv2.circle(dst, (x, y), 5, (0,0,255), -1)

corners2 = corners2.reshape(-1, 2)
for x, y in corners2:    
    cv2.circle(dst, (x, y), 5, (255,0,0), 2)
    
cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()

corners.shape= (8, 1, 2)
corners= [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[168. 270.]]

 [[266. 168.]]

 [[387. 169.]]

 [[108. 269.]]

 [[108. 126.]]]
corners2.shape= (8, 1, 2)
corners2= [[[387. 375.]]

 [[169. 375.]]

 [[265. 126.]]

 [[387. 169.]]

 [[108. 269.]]

 [[266. 168.]]

 [[168. 270.]]

 [[108. 126.]]]


## 2. Chessboard pattern corner point detection

### 2.1 cv2.findChessboardCorners()

In [7]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/chessBoard.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6, 3)
found, corners = cv2.findChessboardCorners(src, patternSize)
print('corners.shape=', corners.shape)

#2
term_crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
corners2 = cv2.cornerSubPix(gray, corners, (5,5), (-1,-1), term_crit)

#3
dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, corners2, found)

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()

corners.shape= (18, 1, 2)


### 2.2 cv2.findCirclesGrid(): Circle pattern center point detection

In [8]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/circleGrid.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
patternSize = (6, 4)
found, centers = cv2.findCirclesGrid(src, patternSize)
print('centers.shape=', centers.shape)

#2
term_crit=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 10, 0.01)
centers2 = cv2.cornerSubPix(gray, centers, (5,5), (-1,-1), term_crit)

#3
dst = src.copy()
cv2.drawChessboardCorners(dst, patternSize, centers2, found)

cv2.imshow('dst',  dst) 
cv2.waitKey()
cv2.destroyAllWindows()

centers.shape= (24, 1, 2)


## 3. Image moments

### 3.1 cv2.moments()

In [9]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/momentTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

#2
##M = cv2.moments(bImage)   
M = cv2.moments(bImage, True)
for key, value in M.items():
    print('{}={}'.format(key, value))
    
#3
cx = int(M['m10']/M['m00'])
cy = int(M['m01']/M['m00'])
dst = src.copy()
cv2.circle(dst, (cx, cy), 5, (0,0,255), 2)

cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()

m00=79262.0
m10=19719561.0
m01=19943644.0
m20=5515429769.0
m11=5090506179.0
m02=5490383844.0
m30=1678594806585.0
m21=1448966367859.0
m12=1427069177889.0
m03=1619623824694.0
mu20=609408144.1012974
mu11=128735034.94252014
mu02=472229671.77704334
mu30=3184447093.48999
mu21=-2863822758.3268433
mu12=-3664976429.7761536
mu03=509725524.1086426
nu20=0.09700144427924574
nu11=0.020491167437841
nu02=0.07516630789606452
nu30=0.001800410227810075
nu21=-0.0016191368967214139
nu12=-0.002072090022265142
nu03=0.00028818662079742156


### 3.2 cv2.moments(): Border moment

In [10]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/circles.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

#2
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
cv2.drawContours(dst, contours, -1, (255,0,0), 3) # all contours

#3
for cnt in contours:
    M = cv2.moments(cnt, True)
##    for key, value in M.items():
##        print('{}={}'.format(key, value))

    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.circle(dst, (cx, cy), 5, (0,0,255), 2)

cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()

### 3.3 invariant moment of Hu

In [11]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/momentTest.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
cnt = contours[0]
cv2.drawContours(dst, [cnt], 0, (255,0,0), 3)

#2
M = cv2.moments(cnt)
hu = cv2.HuMoments(M)
print('hu.shape=', hu.shape)
print('hu=', hu)

#3
angle = 45.0
scale = 0.2
cx = M['m10']/M['m00']
cy = M['m01']/M['m00']
center = (cx, cy)
t = (20, 30)
A = cv2.getRotationMatrix2D( center, angle, scale )
A[:, 2] += t   # translation
print('A=', A) # Affine transformation
cnt2 = cv2.transform(cnt, A)
cv2.drawContours(dst, [cnt2], 0, (0,255,0), 3)
cv2.imshow('dst',  dst)

#4
M2 = cv2.moments(cnt2)
hu2 = cv2.HuMoments(M2)
print('hu2.shape=', hu2.shape)
print('hu2=', hu)

#5
##diffSum = sum(abs(hu - hu2))
diffSum = np.sum(cv2.absdiff(hu, hu2))
print('diffSum=', diffSum)

cv2.waitKey()
cv2.destroyAllWindows()

hu.shape= (7, 1)
hu= [[ 1.72272960e-01]
 [ 2.17960438e-03]
 [ 9.24428655e-05]
 [ 1.90785217e-06]
 [ 1.11977849e-12]
 [-6.96325160e-09]
 [-2.53121609e-11]]
A= [[ 1.41421356e-01  1.41421356e-01  1.98030817e+02]
 [-1.41421356e-01  1.41421356e-01  2.81234993e+02]]
hu2.shape= (7, 1)
hu2= [[ 1.72272960e-01]
 [ 2.17960438e-03]
 [ 9.24428655e-05]
 [ 1.90785217e-06]
 [ 1.11977849e-12]
 [-6.96325160e-09]
 [-2.53121609e-11]]
diffSum= 0.0003215707378081359


## 4. Shape-related feature detection

### 4.1 length, area, bounding square, minimum area square, minimum area circle

In [12]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/banana.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
##bImage = cv2.erode(bImage, None)
bImage = cv2.dilate(bImage, None)
##cv2.imshow('src',  src)
cv2.imshow('bImage',  bImage)

#2
mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)
print('len(contours)=', len(contours))

maxLength = 0
k = 0
for i, cnt in enumerate(contours):
    perimeter = cv2.arcLength(cnt, closed = True)
    if perimeter> maxLength:
        maxLength = perimeter
        k = i
print('maxLength=', maxLength)
cnt = contours[k]
dst2 = src.copy()
cv2.drawContours(dst2, [cnt], 0, (255,0,0), 3)
##cv2.imshow('dst2',  dst2)

#3
area = cv2.contourArea(cnt)
print('area=', area)
x, y, width, height = cv2.boundingRect(cnt)
dst3 = dst2.copy()
cv2.rectangle(dst3, (x, y), (x+width, y+height), (0,0,255), 2)
cv2.imshow('dst3',  dst3)

#4
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int32(box)
print('box=', box)
dst4 = dst2.copy()
cv2.drawContours(dst4,[box],0,(0,0,255),2)
cv2.imshow('dst4',  dst4)

#5
(x,y),radius = cv2.minEnclosingCircle(cnt)
dst5 = dst2.copy()
cv2.circle(dst5,(int(x),int(y)),int(radius),(0,0,255),2)
cv2.imshow('dst5',  dst5)

cv2.waitKey()
cv2.destroyAllWindows()

len(contours)= 1
maxLength= 936.0630499124527
area= 24663.0
box= [[275 391]
 [ 48 147]
 [159  44]
 [386 288]]


### 4.2 straight, polygon, elliptical approximate, and inner points

In [13]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/banana.jpg')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
ret, bImage = cv2.threshold(gray, 220, 255, cv2.THRESH_BINARY_INV)
##bImage = cv2.erode(bImage, None)
bImage = cv2.dilate(bImage, None)
##cv2.imshow('src',  src)
##cv2.imshow('bImage',  bImage)

mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
##cv2.drawContours(dst, contours, -1, (255,0,0), 3)
cnt = contours[0]
cv2.drawContours(dst, [cnt], 0, (255,0,0), 3)

#2
dst2 = dst.copy()
rows,cols = dst2.shape[:2]
[vx,vy,x,y] = cv2.fitLine(cnt, cv2.DIST_L2, 0, 0.01, 0.01)
y1 =  int((-x*vy/vx) + y)
y2 = int(((cols-x)*vy/vx)+y)
cv2.line(dst2,(0,y1), (cols-1,y2),(0,0,255), 2)
cv2.imshow('dst2',  dst2)

#3
ellipse = cv2.fitEllipse(cnt)
dst3 = dst.copy()
cv2.ellipse(dst3, ellipse,(0,0,255),2)
cv2.imshow('dst3',  dst3)

#4
poly = cv2.approxPolyDP(cnt, epsilon=20, closed=True)
dst4 = dst.copy()
cv2.drawContours(dst4, [poly], 0, (0,0,255), 2)
cv2.imshow('dst4',  dst4)

#5
dst5 = dst.copy()
for y in range(rows):
    for x in range(cols):
        if cv2.pointPolygonTest(cnt, (x, y), False)>0:
            dst5[y, x] = (0, 255, 0)            
cv2.imshow('dst5',  dst5)

cv2.waitKey()
cv2.destroyAllWindows()

### 4.3 cv2.convexHull() 

In [14]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/hand.jpg')
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
lowerb = (0, 40, 0)
upperb = (20, 180, 255)
bImage = cv2.inRange(hsv, lowerb, upperb)

mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
##cv2.drawContours(dst, contours, -1, (255,0,0), 3)
cnt = contours[0]
cv2.drawContours(dst, [cnt], 0, (255,0,0), 2)

#2
dst2 = dst.copy()
rows,cols = dst2.shape[:2]
hull = cv2.convexHull(cnt)
cv2.drawContours(dst2, [hull], 0, (0,0,255), 2)
cv2.imshow('dst2',  dst2)

cv2.waitKey()
cv2.destroyAllWindows()

### 4.4 cv2.convexityDefects()

In [15]:
import cv2
import numpy as np

#1
src = cv2.imread('./data/hand.jpg')
hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
lowerb = (0, 40, 0)
upperb = (20, 180, 255)
bImage = cv2.inRange(hsv, lowerb, upperb)

mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, contours, hierarchy = cv2.findContours(bImage, mode, method)

dst = src.copy()
##cv2.drawContours(dst, contours, -1, (255,0,0), 3)
cnt = contours[0]
cv2.drawContours(dst, [cnt], 0, (255,0,0), 2)

#2
dst2 = dst.copy()
rows,cols = dst2.shape[:2]
hull = cv2.convexHull(cnt, returnPoints = False)
hull_points = cnt[hull[:,0]]
cv2.drawContours(dst2, [hull_points], 0, (255,0,255), 6)

#3
T = 5 # 5, 10
defects = cv2.convexityDefects(cnt, hull)
print('defects.shape=',defects.shape)
for i in range(defects.shape[0]):
    s,e,f,d = defects[i,0]
    dist = d/256
    start = tuple(cnt[s][0])
    end   = tuple(cnt[e][0])
    far   = tuple(cnt[f][0])    
    if dist > T:
        cv2.line(dst2,start,end,[255,255,0],2)
        cv2.line(dst2,start,far,[0,255,0],1)
        cv2.line(dst2,end,  far,[0,255,0],1)
        
        cv2.circle(dst2,start,5,[0,255,255],-1)
        cv2.circle(dst2,end,  5,[0,128,255],-1)
        cv2.circle(dst2,far,  5,[0,0,255],-1)   
cv2.imshow('dst2',  dst2)

cv2.waitKey()
cv2.destroyAllWindows()

defects.shape= (25, 1, 4)


## 5. Shape matching

### 5.1 cv2.matchShapes()

In [16]:
import cv2
import numpy as np

#1
ref_src  = cv2.imread('./data/refShapes.jpg')
ref_gray = cv2.cvtColor(ref_src, cv2.COLOR_BGR2GRAY)
ret, ref_bin = cv2.threshold(ref_gray, 0, 255,
                             cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

test_src  = cv2.imread('./data/testShapes1.jpg')
##test_src  = cv2.imread('./data/testShapes2.jpg')
##test_src  = cv2.imread('./data/testShapes3.jpg')
test_gray = cv2.cvtColor(test_src, cv2.COLOR_BGR2GRAY)
ret, test_bin = cv2.threshold(test_gray, 0, 255,
                             cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
mode   = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
image, ref_contours, hierarchy = cv2.findContours(ref_bin, mode, method)
image,test_contours, hierarchy = cv2.findContours(test_bin, mode, method)

#2
ref_dst = ref_src.copy()
colors = ((0,0,255), (0,255,0), (255,0,0))
for i, cnt in enumerate(ref_contours):
    cv2.drawContours(ref_dst, [cnt], 0, colors[i], 2)

#3: shape matching
test_dst = test_src.copy()
method = cv2.CONTOURS_MATCH_I1    
for i, cnt1 in enumerate(test_contours):
    matches = []
    for cnt2 in ref_contours:
        ret = cv2.matchShapes(cnt1, cnt2, method, 0)
        matches.append(ret)
    k = np.argmin(matches)
    cv2.drawContours(test_dst, [cnt1], 0, colors[k], 2)
       
cv2.imshow('ref_dst',  ref_dst)
cv2.imshow('test_dst', test_dst)

cv2.waitKey()
cv2.destroyAllWindows()

## 6. Integral image

### 6.1 cv2.integral()

In [17]:
import cv2
import numpy as np

#1
A = np.arange(1, 17).reshape(4, 4).astype(np.uint8)
print('A=', A)

#2
sumA, sqsumA, tiltedA = cv2.integral3(A)
print('sumA=', sumA)
print('sqsumA=', np.uint32(sqsumA))
print('tiltedA=', tiltedA)

A= [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]
 [13 14 15 16]]
sumA= [[  0   0   0   0   0]
 [  0   1   3   6  10]
 [  0   6  14  24  36]
 [  0  15  33  54  78]
 [  0  28  60  96 136]]
sqsumA= [[   0    0    0    0    0]
 [   0    1    5   14   30]
 [   0   26   66  124  204]
 [   0  107  247  426  650]
 [   0  276  612 1016 1496]]
tiltedA= [[ 0  0  0  0  0]
 [ 0  1  2  3  4]
 [ 1  8 12 16 15]
 [ 8 26 38 42 36]
 [26 60 80 84 70]]


### 6.2 cv2.integral()

In [18]:
import cv2
import numpy as np

#1
gray = cv2.imread('./data/lena.jpg', cv2.IMREAD_GRAYSCALE)

#2
gray_sum = cv2.integral(gray)
dst = cv2.normalize(gray_sum, None, 0, 255,
                    cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imshow('dst',  dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 7. Haar-like feature

### 7.1 Haar-like feature 1

In [19]:
import cv2
import numpy as np
#1
def rectSum(sumImage, rect):
    x, y, w, h = rect
    a = sumImage[y, x]
    b = sumImage[y, x+w]
    c = sumImage[y+h, x]
    d = sumImage[y+h, x+w]
    return a + d - b - c
#2
def compute_Haar_feature1(sumImage, rect):
    x, y, w, h = rect
##    print(x, y, w, h)
    s1 = rectSum(sumImage, (x,  y, w, h))
    s2 = rectSum(sumImage, (x+w,y, w, h))
##    print('s1=', s1)
##    print('s2=', s2)    
    return s1-s2
def compute_Haar_feature2(sumImage, rect):
    x, y, w, h = rect
    s1 = rectSum(sumImage, (x,  y, w, h))
    s2 = rectSum(sumImage, (x,y+h, w, h)) 
    return s2-s1
def compute_Haar_feature3(sumImage, rect):
    x, y, w, h = rect
    s1 = rectSum(sumImage, (x,    y, w, h))
    s2 = rectSum(sumImage, (x+w,  y, w, h))
    s3 = rectSum(sumImage, (x+2*w,y, w, h))
    return s1-s2+s3
def compute_Haar_feature4(sumImage, rect):
    x, y, w, h = rect
    s1 = rectSum(sumImage, (x,    y,  w, h))
    s2 = rectSum(sumImage, (x,  y+h,  w, h))
    s3 = rectSum(sumImage, (x,  y+2*h,w, h))
    return s1-s2+s3
def compute_Haar_feature5(sumImage, rect):
    x, y, w, h = rect
    s1 = rectSum(sumImage, (x,    y, w, h))
    s2 = rectSum(sumImage, (x+w,  y, w, h))
    s3 = rectSum(sumImage, (x,  y+h, w, h))
    s4 = rectSum(sumImage, (x+w,y+h, w, h))
    return s1+s4-s2-s3

#3
A = np.arange(1, 6*6+1).reshape(6, 6).astype(np.uint8)
print('A=', A)

h, w = A.shape
sumA = cv2.integral(A)
print('sumA=', sumA)

#4
f1 = compute_Haar_feature1(sumA, (0, 0, w//2, h))    # 3, 6
print('f1=', f1)

#5
f2 = compute_Haar_feature2(sumA, (0, 0, w, h//2))    # 6, 3
print('f2=', f2)

#6
f3 = compute_Haar_feature3(sumA, (0, 0, w//3, h))    # 2, 6
print('f3=', f3)

#7
f4 = compute_Haar_feature4(sumA, (0, 0, w, h//3))    # 6, 2
print('f4=', f4)

#8
f5 = compute_Haar_feature5(sumA, (0, 0, w//2, h//2)) # 3, 3
print('f5=', f5)

A= [[ 1  2  3  4  5  6]
 [ 7  8  9 10 11 12]
 [13 14 15 16 17 18]
 [19 20 21 22 23 24]
 [25 26 27 28 29 30]
 [31 32 33 34 35 36]]
sumA= [[  0   0   0   0   0   0   0]
 [  0   1   3   6  10  15  21]
 [  0   8  18  30  44  60  78]
 [  0  21  45  72 102 135 171]
 [  0  40  84 132 184 240 300]
 [  0  65 135 210 290 375 465]
 [  0  96 198 306 420 540 666]]
f1= -54
f2= 324
f3= 222
f4= 222
f5= 0


### 7.2 Haar-like feature 2: all possible features

In [20]:
import cv2
import numpy as np

#1
def rectSum(sumImage, rect):
    x, y, w, h = rect
    a = sumImage[y, x]
    b = sumImage[y, x+w]
    c = sumImage[y+h, x]
    d = sumImage[y+h, x+w]
    return a + d - b - c
def compute_Haar_feature1(sumImage):
    rows, cols = sumImage.shape
    rows -= 1
    cols -= 1
    f1 = []
    for y in range(0, rows):
        for x in range(0, cols):
            for h in range(1, rows-y+1):
                for w in range(1, (cols-x)//2+1):
                    s1 = rectSum(sumImage, (x,  y, w, h))
                    s2 = rectSum(sumImage, (x+w,y, w, h))
                    f1.append([1, x, y, w, h, s1-s2])    
    return f1
def compute_Haar_feature2(sumImage):
    rows, cols = sumImage.shape
    rows -= 1
    cols -= 1
    f2 = []
    for y in range(0, rows):
        for x in range(0, cols):
            for h in range(1, (rows-y)//2+1):
                for w in range(1, cols-x+1):
                    s1 = rectSum(sumImage, (x,  y, w, h))
                    s2 = rectSum(sumImage, (x,y+h, w, h))
                    f2.append([2, x, y, w, h, s2-s1])    
    return f2
def compute_Haar_feature3(sumImage):
    rows, cols = sumImage.shape
    rows -= 1
    cols -= 1
    f3 = []
    for y in range(0, rows):
        for x in range(0, cols):
            for h in range(1, rows-y+1):
                for w in range(1, (cols-x)//3+1):
                    s1 = rectSum(sumImage, (x,    y, w, h))
                    s2 = rectSum(sumImage, (x+w,  y, w, h))
                    s3 = rectSum(sumImage, (x+2*w,y, w, h))                    
                    f3.append([3, x, y, w, h, s1-s2+s3])    
    return f3
def compute_Haar_feature4(sumImage):
    rows, cols = sumImage.shape
    rows -= 1
    cols -= 1
    f4 = []
    for y in range(0, rows):
        for x in range(0, cols):
            for h in range(1, (rows-y)//3+1):
                for w in range(1, cols-x+1):
                    s1 = rectSum(sumImage, (x,  y,   w, h))
                    s2 = rectSum(sumImage, (x,y+h,   w, h))
                    s3 = rectSum(sumImage, (x,y+2*h, w, h))
                    f4.append([4, x, y, w, h, s1-s2+s3])    
    return f4
def compute_Haar_feature5(sumImage):
    rows, cols = sumImage.shape
    rows -= 1
    cols -= 1
    f5 = []
    for y in range(0, rows):
        for x in range(0, cols):
            for h in range(1, (rows-y)//2+1):
                for w in range(1, (cols-x)//2+1):
                    s1 = rectSum(sumImage, (x,  y,   w, h))
                    s2 = rectSum(sumImage, (x+w,y,   w, h))
                    s3 = rectSum(sumImage, (x,  y+h, w, h))
                    s4 = rectSum(sumImage, (x+w,y+h, w, h))
                    f5.append([5, x, y, w, h, s1-s2-s3+s4])    
    return f5

#2
gray = cv2.imread('./data/lenaFace24.jpg', cv2.IMREAD_GRAYSCALE) # 24 x 24
gray_sum = cv2.integral(gray)
f1 = compute_Haar_feature1(gray_sum)
n1 = len(f1)
print('len(f1)=',n1)
for i, a in enumerate(f1[:2]):
    print('f1[{}]={}'.format(i, a))
#3
f2 = compute_Haar_feature2(gray_sum)
n2 = len(f2)
print('len(f2)=',n2)
for i, a in enumerate(f2[:2]):
    print('f2[{}]={}'.format(i, a))

#4
f3 = compute_Haar_feature3(gray_sum)
n3 = len(f3)
print('len(f3)=',n3)
for i, a in enumerate(f3[:2]):
    print('f3[{}]={}'.format(i, a))

#5
f4 = compute_Haar_feature4(gray_sum)
n4 = len(f4)
print('len(f4)=',n4)
for i, a in enumerate(f4[:2]):
    print('f4[{}]={}'.format(i, a))
#6
f5 = compute_Haar_feature5(gray_sum)
n5 = len(f5)
print('len(f5)=',n5)
for i, a in enumerate(f5[:2]):
    print('f5[{}]={}'.format(i, a))
    
print('total features =', n1+n2+n3+n4+n5)

len(f1)= 43200
f1[0]=[1, 0, 0, 1, 1, -11]
f1[1]=[1, 0, 0, 2, 1, 6]
len(f2)= 43200
f2[0]=[2, 0, 0, 1, 1, 25]
f2[1]=[2, 0, 0, 2, 1, 6]
len(f3)= 27600
f3[0]=[3, 0, 0, 1, 1, 138]
f3[1]=[3, 0, 0, 2, 1, 324]
len(f4)= 27600
f4[0]=[4, 0, 0, 1, 1, 170]
f4[1]=[4, 0, 0, 2, 1, 373]
len(f5)= 20736
f5[0]=[5, 0, 0, 1, 1, -44]
f5[1]=[5, 0, 0, 2, 1, 25]
total features = 162336
